In [1]:
import sys
import pandas as pd
import dateutil
import numpy as np
sys.path.append('../')
from utilities.era5_down import era5_down
import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
#cs_file = '../data/SMB_bellin_diff_1.csv'
#cs_file = '../data/SMB_bellin_season.csv'
cs_file = '../data/SMB_bellin_diff.csv'

dfmb = pd.read_csv(cs_file,
   delimiter='\t', parse_dates=['Date', 'Date1'], na_values='NAN')
labels = dfmb.drop_duplicates(subset=['Stake'])

In [5]:
t2m = 't2m'
Snowfall = 'sf'

In [6]:
files = sorted(glob.glob('../data/ERA_59_20_day/*.nc'))
len(files)

62

In [7]:
data_pro = []
for st in labels['Stake']:
    stake  = dfmb.loc[lambda df: df['Stake'] == st, :].copy().reset_index()
    df_day = era5_down(files, stake['Longitude'][0],
                       stake['Latitude'][0], 
                       stake['Elevation'][0])
    subset_f0 = []
    for i in range(len(stake)):
        subset = df_day.loc[str(stake.Date1[i]): str(stake.Date[i])].copy()
        subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0)
        subset['stake'] = stake['Stake'][0]
        subset_f = subset.groupby(['stake']).agg({t2m:np.mean, Snowfall:nansumwrapper, 'PDD':nansumwrapper})
        subset_f.index = ([stake.Date[i]])
        subset_f0.append(subset_f)
    
    data_pro.append(pd.concat([stake, pd.concat(subset_f0).reset_index()], axis=1))

In [8]:
data_inp_four = pd.concat(data_pro)
data_inp_four.drop(['index', 'Date1', 'index'], axis=1, inplace=True)

In [9]:
data_inp_four.columns

Index(['Stake', 'Date', 'Latitude', 'Longitude', 'Elevation', 'Days', 'SMB',
       't2m', 'sf', 'PDD'],
      dtype='object')

In [10]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,sf,PDD
0,R15,2008-01-05,-62.178873,-58.907921,54.0,0,0.49203,-0.371277,0.000085,0.000000
1,R15,2008-01-26,-62.178873,-58.907921,54.0,21,-0.23003,0.669618,0.018224,15.887604
2,R15,2008-02-02,-62.178873,-58.907921,54.0,7,-0.02890,1.335468,0.000028,10.683746
3,R15,2008-02-09,-62.178873,-58.907921,54.0,7,-0.04356,0.495777,0.009557,4.213654
4,R15,2008-02-17,-62.178873,-58.907921,54.0,8,-0.20554,1.629045,0.000120,14.661407
...,...,...,...,...,...,...,...,...,...,...
80,9N,2012-02-04,-62.166223,-58.883543,261.0,15,-0.07890,-1.191734,0.007514,0.085175
81,9N,2012-02-19,-62.166223,-58.883543,261.0,15,-0.08170,-1.147125,0.012092,1.320099
82,9N,2012-03-04,-62.166223,-58.883543,261.0,14,0.09720,-2.062679,0.004308,0.260529
83,9N,2012-03-17,-62.166223,-58.883543,261.0,13,-0.21880,-0.534182,0.002368,1.038239


In [11]:
# reset SMB, tp, snowfall and PDD to 0 on 0 days observation
#data_inp_four[['SMB',Prec, Snowfall, 'PDD' ]] = data_inp_four[['SMB',Prec, Snowfall,
#                                                               'PDD' ]].where(data_inp_four['Days'] <= 100,
#                                                                              np.nan)
data_inp_four[['SMB', Snowfall, 'PDD' ]] = data_inp_four[['SMB', Snowfall,'PDD' ]].where(data_inp_four['Days'] != 0, np.nan)

#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['SMB'] < 0, 0)
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 260, 0) # four
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 200, 0) # bell


data_inp_four.dropna(inplace=True)

In [12]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,sf,PDD
1,R15,2008-01-26,-62.178873,-58.907921,54.0,21,-0.23003,0.669618,0.018224,15.887604
2,R15,2008-02-02,-62.178873,-58.907921,54.0,7,-0.02890,1.335468,0.000028,10.683746
3,R15,2008-02-09,-62.178873,-58.907921,54.0,7,-0.04356,0.495777,0.009557,4.213654
4,R15,2008-02-17,-62.178873,-58.907921,54.0,8,-0.20554,1.629045,0.000120,14.661407
5,R15,2008-02-24,-62.178873,-58.907921,54.0,7,-0.11200,1.876850,0.001954,15.014801
...,...,...,...,...,...,...,...,...,...,...
80,9N,2012-02-04,-62.166223,-58.883543,261.0,15,-0.07890,-1.191734,0.007514,0.085175
81,9N,2012-02-19,-62.166223,-58.883543,261.0,15,-0.08170,-1.147125,0.012092,1.320099
82,9N,2012-03-04,-62.166223,-58.883543,261.0,14,0.09720,-2.062679,0.004308,0.260529
83,9N,2012-03-17,-62.166223,-58.883543,261.0,13,-0.21880,-0.534182,0.002368,1.038239


In [13]:
data_inp_four.to_csv('../data/SMB_input_belli_ERA5.csv', index=False, sep='\t')